In [8]:
from IPython.display import display
import ipywidgets as widgets
from os import listdir
from os.path import isfile, join
import re
import numpy as np
from enum import Enum
from copy import deepcopy
from pprint import pprint
from functools import reduce

files = [f for f in listdir('.') if isfile(join('.', f))]
pat = re.compile('(ex(\d)+.txt)|(in(\d)*.txt)')
files = [f for f in files if pat.match(f)]

box = widgets.Select(
    options=files,
    value='ex1.txt',
    rows=4,
    description='Input file:',
    disabled=False
)

display(box)


Select(description='Input file:', index=1, options=('in.txt', 'ex1.txt'), rows=4, value='ex1.txt')

## Parse file

In [15]:
name = box.value
f = open(name, 'r')
lines = f.readlines()
lines = list(map(lambda l: l.rstrip(), lines))


# TODO: Parse the input

class Tree:
    c = 1
    nodes = {}
    curr = 0

    def __init__(self):
        self.nodes[0] = Node('/', 0, [], [])

    def add_dir(self, name):
        self.nodes[self.curr].children.append(self.c)
        self.nodes[self.c] = Node(name, self.curr, [], [])
        self.c += 1

    def add_file(self, name, size):
        self.nodes[self.curr].files.append((name, size))

    def cd(self, path):
        if path == '/':
            self.curr = 0
        elif path == '..':
            self.curr = self.nodes[self.curr].parent
        else:
            for child in self.nodes[self.curr].children:
                if self.nodes[child].name == path:
                    self.curr = child
                    break


class Node:
    def __init__(self, name, parent, children, files):
        self.name = name
        self.parent = parent
        self.children = children
        self.files = files

    def __repr__(self):
        return "Node({name}, {parent}, {children}, {files})".format(name=self.name, parent=self.parent, children=self.children, files=self.files)


tree = Tree()
i = 0
while i < len(lines):
    line = lines[i]
    if line[2:4] == 'cd':
        tree.cd(line[5:])
        i += 1
    elif line[2:4] == 'ls':
        j = 1
        while i + j < len(lines) and lines[i + j][0] != '$':
            l = lines[i + j]
            if l[:3] == 'dir':
                tree.add_dir(l[4:])
            else:
                t = l.split()
                tree.add_file(t[1], int(t[0]))
            j += 1
        i += j


print("Done parsing \"{name}\"".format(name=name))


Done parsing "in.txt"


### Part 1

In [10]:
def count(node):
    c = 0
    stack = [node]
    while len(stack) > 0:
        n = stack.pop()
        c += sum([f[1] for f in tree.nodes[n].files])
        stack.extend(tree.nodes[n].children)

    return c

t = 0
for node in tree.nodes:
    c = count(node)
    if c <= 100_000:
        t += c

print(t)

95437


### Part 2

In [16]:
MAX = 40_000_000

x = count(0) - MAX
D = []
for node in tree.nodes:
    c = count(node)
    if c >= x:
        D.append((node, c))
        
D.sort(key=lambda d: d[1])

print(D[0][1])


7268994
